In [4]:
from gensim.test.utils import get_tmpfile
import pandas as pd
import numpy as np
import itertools
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
import pickle
import regex as re
from keras.models import Sequential
from keras import layers
from keras.layers import Embedding, Flatten, Dense, Dropout, LSTM
from keras.utils.np_utils import to_categorical
from sklearn.metrics import confusion_matrix

c:\users\thora\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


### Create Twitter Character Embeddings from twcs.csv

In [2]:
import pandas as pd
df = pd.read_csv('twcs.csv', sep=',', encoding='utf8')

In [3]:
texts=list(df['text'])

In [5]:
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [13]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

# Function takes a tokenized sentence and returns the words
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

# Creating texts 
t = list(map(sequence_to_text, sequences))

In [5]:
len(t)

In [2]:
from gensim.models import FastText

path = get_tmpfile("chars_twitter.model")

model = FastText(t, size=300, min_count=1, workers=4)
model.save("chars_twitter.model")

### Fine tune the Embeddings

In [3]:
model = FastText.load("chars_twitter.model")

In [1]:
def load_data(csv, p1, p2):
    """
    Loads data from 3 specific paths. 
    Returns a list of texts, a corresponding list of labels (0|1|2), and a labels counter.
    """
    df = pd.read_csv(csv, index_col=0, encoding = 'utf-8')

    texts = []
    labels = []
    for row in df.itertuples(index=True):
        tweet = getattr(row, "tweet")
        classif = getattr(row, "classification")
        texts.append(tweet)
        labels.append(classif)

    more_hateful_tweets = pickle.load(open(p1, "rb" ))
    more_clean_tweets = pickle.load(open(p2, "rb" ))

    hateful_labels = [0] * len(more_hateful_tweets)
    clean_labels = [2] * len(more_clean_tweets)
    print('Additional Hateful:', len(more_hateful_tweets), 'Additional Clean:', len(more_clean_tweets))
    
    texts = texts+more_hateful_tweets+more_clean_tweets
    labels = labels+hateful_labels+clean_labels
    cnt = Counter(labels)
    print(cnt)
    return texts, labels, cnt

In [2]:
from collections import Counter

In [6]:
texts, labels, cnt = load_data('../../Data/Data_Original/offensive_language_crowdflower.csv', "../../Data/Data_Original/tweets_hate_icwsm18_extended.p", "../../Data/Data_Original/tweets_clean_zeerakW_extended.p")

Additional Hateful: 9802 Additional Clean: 3017
Counter({1: 19190, 0: 11232, 2: 7180})


In [7]:
def tokenize_texts_characters(corpus):
    """
    Tokenizes a list of texts. ...!
    """
    corpus = convert_html_emojis(corpus)
    
    tknzr = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)
    
    
    token_doc_list = []
    token_list = []
   
    for text in corpus:
     

        tokens = tknzr.tokenize(text)
        tokens2 = []
        for token in tokens:
        #    token = re.sub(r'([1234567890!"#$%&()*+,./:@;?[\]^`{|}_~\t\n])', '', token)
            if token != '' and token !='charlie' and token !='yankees':
                tokens2.append(token)
        

        
        tmp_tokens = []
        for i, token in enumerate(tokens2):
            #if token in UNICODE_EMOJI:
               
                #emoji_one = emoji.demojize(token)
                #emoji_one = emoji_one[1:-1]
                #emoji_all = re.split('_', emoji_one)

                #tokens[i] = emoji_all
                #token_list.extend(emoji_all)
                #tmp_tokens.extend(emoji_all)
            if token == "i'd":
                token_list.extend(['i', 'would'])
                tmp_tokens.extend(['i', 'would'])
            else: 
                token_list.append(token)
                tmp_tokens.append(token)

            
        token_doc_list.append(tmp_tokens)
        
    final_texts = []
    for doc in token_doc_list:
        t = " ".join(doc)
        final_texts.append(t)
    return final_texts

In [8]:
import html
def convert_html_emojis(corpus):
    new_corpus = []
    for text in corpus:
        text = html.unescape(text)
        new_corpus.append(text)
    return new_corpus

In [9]:
from nltk.tokenize import TweetTokenizer
from emoji import UNICODE_EMOJI
texts = tokenize_texts_characters(texts)

In [10]:
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [14]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
d = list(map(sequence_to_text, sequences))

In [15]:
len(d)

37602

In [16]:
d[:10]

[['!',
  ' ',
  '!',
  ' ',
  '!',
  ' ',
  'r',
  't',
  ' ',
  ':',
  ' ',
  'a',
  's',
  ' ',
  'a',
  ' ',
  'w',
  'o',
  'm',
  'a',
  'n',
  ' ',
  'y',
  'o',
  'u',
  ' ',
  's',
  'h',
  'o',
  'u',
  'l',
  'd',
  'n',
  "'",
  't',
  ' ',
  'c',
  'o',
  'm',
  'p',
  'l',
  'a',
  'i',
  'n',
  ' ',
  'a',
  'b',
  'o',
  'u',
  't',
  ' ',
  'c',
  'l',
  'e',
  'a',
  'n',
  'i',
  'n',
  'g',
  ' ',
  'u',
  'p',
  ' ',
  'y',
  'o',
  'u',
  'r',
  ' ',
  'h',
  'o',
  'u',
  's',
  'e',
  ' ',
  '.',
  ' ',
  '&',
  ' ',
  'a',
  's',
  ' ',
  'a',
  ' ',
  'm',
  'a',
  'n',
  ' ',
  'y',
  'o',
  'u',
  ' ',
  's',
  'h',
  'o',
  'u',
  'l',
  'd',
  ' ',
  'a',
  'l',
  'w',
  'a',
  'y',
  's',
  ' ',
  't',
  'a',
  'k',
  'e',
  ' ',
  't',
  'h',
  'e',
  ' ',
  't',
  'r',
  'a',
  's',
  'h',
  ' ',
  'o',
  'u',
  't',
  ' ',
  '.',
  '.',
  '.'],
 ['!',
  ' ',
  '!',
  ' ',
  '!',
  ' ',
  'r',
  't',
  ' ',
  ':',
  ' ',
  'b',
  'o',
  'y',
  ' ',
  'd'

In [17]:
model.build_vocab(d, update=True)
model.train(d, total_examples=len(d), epochs=20)
model.save("ft_chars_twitter_finetuned_20_allchars.model")

In [19]:
#https://radimrehurek.com/gensim/models/fasttext.html#gensim.models.fasttext.FastText